In [1]:
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font
from openpyxl.comments import Comment
import pandas as pd
from utilities import *

In [36]:
def compare_excels(ref_path, new_path, output_path):
    try :
        df_old_version = pd.read_excel(ref_path)
        df_new_version = pd.read_excel(new_path)

        if df_old_version.shape != df_new_version.shape:
            print("Error: DataFrames have different shapes. Cannot compare.")
            return 
            
        wb_new = load_workbook(new_path)
        ws_new = wb_new.active

        if "Summary" in wb_new.sheetnames:
            wb_new.remove(wb_new["Summary"])
        ws_summary = wb_new.create_sheet("Summary")

        # Ajouter des en-têtes au résumé
        ws_summary.append(["Type", "Row/Column", "Old Value", "New Value"])

        red_fill = PatternFill(start_color='FFFF0000', end_color='FFFF0000', fill_type='solid')
        green_fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')
        font = Font(color='FFFFFF')

        old_columns = set(df_old_version.columns)
        new_columns = set(df_new_version.columns)
        
        added_columns = new_columns - old_columns
        removed_columns = old_columns - new_columns
        
        for col in added_columns:
            ws_summary.append(["Added Column", col, "None", "Exists"])
            ws_new.cell(row=1, column= added_columns.index(col)).fill = green_fill
        
        for col in removed_columns:
            ws_summary.append(["Removed Column", col, "Exists", "None"])
            ws_new.cell(row=1, column= removed_columns.index(col)).fill = red_fill
        
        # Comparaison des valeurs cellule par cellule
        common_columns = list(old_columns & new_columns)
        df_old_version = df_old_version[common_columns]
        df_new_version = df_new_version[common_columns]

        for row_index in range(0, df_new_version.shape[0]):
            for col_index in range(1, df_new_version.shape[1]):
                old_value = df_old_version.iloc[row_index,col_index]
                ref_value = df_new_version.iloc[row_index,col_index]
                # print(f'old value {old_value} and new value {ref_value}')
        
                if pd.isna(old_value) and pd.isna(ref_value):                
                    continue
                elif str(old_value) != str(ref_value) or (pd.isna(old_value) or pd.isna(ref_value)):
                    print(f'old value {old_value} ,new value {ref_value}')
                    cell = ws_new.cell(row=row_index + 2, column=col_index+1)
                    cell.fill = red_fill
                    cell.font = font
                    cell.comment = Comment(add_comment(old_value, ref_value), "AutoComparer")
                    if pd.isna(old_value):
                        old_value = 'None'
                    elif pd.isna(ref_value):
                        ref_value = 'None'
                    # ws_summary.append(["Value Change", f"Row {row_index + 2}, Col {col_name}", old_value, new_value])
                    # ws_summary.append([row_index + 2, df_new_version.columns[col_index], old_value, ref_value])
                    
                    
        wb_new.save(output_path)
        print(f"Comparison complete. Differences highlighted in '{output_path}', summary added in 'Summary' sheet.")
    except FileNotFoundError:
        print("Error: One or both of the Excel files were not found.")
    except Exception as e:
        print(f"An error occurred: {e}")        

In [150]:
ref_path = "yazaki_data/ref.xlsx"
new_path ="yazaki_data/new.xlsx"
output_path = 'yazaki_data/output.xlsx'

In [151]:
compare_excels(ref_path, new_path, output_path)

Row index 2,col index : 53 old value X ,new value XX
Row index 3,col index : 14 old value SQ ,new value SQA
Row index 5,col index : 7 old value L/W ,new value L/Wx
Comparison complete. Differences highlighted in 'yazaki_data/output.xlsx', summary added in 'Summary' sheet.


In [149]:
def compare_excels(ref_path, new_path, output_path):
    try:
        df_old_version = pd.read_excel(ref_path)
        df_new_version = pd.read_excel(new_path)
        
        wb_new = load_workbook(new_path)
        ws_new = wb_new.active

        if "Summary" in wb_new.sheetnames:
            wb_new.remove(wb_new["Summary"])
        ws_summary = wb_new.create_sheet("Summary")

        # Ajouter des en-têtes au résumé
        ws_summary.append(["Type", "Row/Column", "Old Value", "New Value"])

        red_fill = PatternFill(start_color='FFFF0000', end_color='FFFF0000', fill_type='solid')
        green_fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')
        font = Font(color='FFFFFF')

        # Détection des colonnes ajoutées ou supprimées
        old_columns = set(df_old_version.columns)
        new_columns = set(df_new_version.columns)
        
        added_columns = new_columns - old_columns
        removed_columns = old_columns - new_columns
        
        for col in added_columns:
            ws_summary.append(["Added Column", col, "None", "Exists"])
        
        for col in removed_columns:
            ws_summary.append(["Removed Column", col, "Exists", "None"])
        
        # Comparaison des valeurs cellule par cellule
        common_columns = list(old_columns & new_columns)
        # df_old_version = df_old_version[common_columns]
        # df_new_version = df_new_version[common_columns]


        
        for row_index in range(0, min(df_old_version.shape[0], df_new_version.shape[0])):
            for col_name in common_columns:
                if col_name in old_columns:
                    col_index = list(df_old_version.columns).index(col_name)
                else :
                    col_index = list(df_new_version.columns).index(col_name)
                old_value = df_old_version.iloc[row_index, col_index]
                new_value = df_new_version.iloc[row_index, col_index]
                
                if pd.isna(old_value) and pd.isna(new_value):
                    continue
                elif (str(old_value) != str(new_value)):
                    print(f'Row index {row_index + 2},col index : {col_index} old value {old_value} ,new value {new_value}')
                    cell = ws_new.cell(row=row_index + 2, column=col_index + 1)
                    cell.fill = red_fill
                    cell.font = font
                    cell.comment = Comment(add_comment(old_value, new_value), "AutoComparer")
                    if pd.isna(old_value):
                        old_value = 'None'
                    elif pd.isna(new_value):
                        new_value = 'None'
                    ws_summary.append(["Value Change", f"Row {row_index + 2}, Col {col_name}", old_value, new_value])

        wb_new.save(output_path)
        print(f"Comparison complete. Differences highlighted in '{output_path}', summary added in 'Summary' sheet.")
    except FileNotFoundError:
        print("Error: One or both of the Excel files were not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [134]:
df_old_version = pd.read_excel(ref_path)
df_new_version = pd.read_excel(new_path)

In [135]:
old_columns = set(df_old_version.columns)
new_columns = set(df_new_version.columns)
added_columns = new_columns - old_columns
removed_columns = old_columns - new_columns

In [145]:
'To Eyelet Protection' in list(df_old_version.columns)

False

In [144]:
'To Eyelet Protection' in list(df_new_version.columns)

True

In [143]:
added_columns

{'To Eyelet Protection'}